<a href="https://colab.research.google.com/github/aomidvar/scrapper/blob/main/Symantec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import re




def extract():
    a = []
    req = requests.get("https://symantec-enterprise-blogs.security.com/blogs/threat-intelligence")
    html_page = req.text

    soup = BeautifulSoup(html_page, "lxml")

    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))
    for link in links:
        if type(link) == str :
            if '/threat-intelligence/' in link :
                a.append(link)
    return a

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
s = '/blog-post/solarwinds-raindrop-malware'

a = s.split('/')
print(type(a[-1::]))

<class 'list'>


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import requests
import bs4 as bs
from bs4 import BeautifulSoup
import json

def extract():
    a = []
    req = requests.get("https://symantec-enterprise-blogs.security.com/blogs/threat-intelligence")
    html_page = req.text

    soup = BeautifulSoup(html_page, "lxml")

    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))
    for link in links:
        if type(link) == str :
            if '/threat-intelligence/' in link :
                a.append(link)
    return a

def crawl(link):
    r = requests.get('https://symantec-enterprise-blogs.security.com/blogs/threat-intelligence/'+link)
    parsed_article = bs.BeautifulSoup(r.text, 'lxml')
    paragraphs = parsed_article.find_all('p')
    article_text = ""

    for p in paragraphs:
        article_text += p.text
    processed_article = article_text.lower()

    return processed_article

if __name__ == '__main__':
    lis_of_json = []
    a = extract()
    for link in a :
        s = link.split('/')
        titr = (s[len(s)-1])
        tozihat = crawl(titr)
        dict = {
            'Name' : titr,
            'Explanation' : tozihat
        }
        lis_of_json.append(dict)
    json_object = json.dumps(lis_of_json, indent=4,ensure_ascii=False)
    with open("/content/drive/MyDrive/symantec.json", "a") as outfile:
        outfile.write(json_object)



In [ ]:
import requests
import json
import bs4 as bs
from bs4 import BeautifulSoup


def crawl(link):
    r = requests.get('https://symantec-enterprise-blogs.security.com/blogs/threat-intelligence/'+link)
    parsed_article = bs.BeautifulSoup(r.text, 'lxml')
    paragraphs = parsed_article.find_all('p')
    article_text = ""

    for p in paragraphs:
        article_text += p.text
    processed_article = article_text.lower()

    return processed_article


if __name__ == '__main__':
    list_of_cti = []
    end = False
    page = 1
    while not end:
        r = requests.get('https://symantec-enterprise-blogs.security.com/blogs/api/v1/blogs/search?aid=IOTVy1&blog=221&sort=&sortDirection=&rows=1&page='+str(page))
        json_object = json.loads(r.text)
        if len(json_object['results']) == 0 :
            end =True
        else :
            urltitle = json_object['results'][0]['urlAlias']
            title = json_object['results'][0]['title']
            tmp = urltitle.split('/')
            url_to_crawl = tmp[len(tmp)-1]
            cti = crawl(url_to_crawl)
            to_file = {
                'tite' : title,
                'Explanation' : cti
            }
            list_of_cti.append(to_file)
            print(to_file)
        page = page + 1

    with open("/content/drive/MyDrive/symantec.json", "w",encoding='utf-8') as outfile:
        json.dump(list_of_cti,outfile,ensure_ascii=False,indent=4)

{'tite': 'Crambus: New Campaign Targets Middle Eastern Government', 'Explanation': 'the iranian crambus espionage group (aka oilrig, apt34) staged an eight-month-long intrusion against a government in the middle east between february and september 2023. during the compromise, the attackers stole files and passwords and, in one case, installed a powershell backdoor (dubbed powerexchange) that was used to monitor incoming mails sent from an exchange server in order to execute commands sent by the attackers in the form of emails, and surreptitiously forwarded results\xa0 to the attackers. malicious activity occurred on at least 12 computers and there is evidence that the attackers deployed backdoors and keyloggers on dozens more.in addition to deploying malware, the attackers made frequent use of the publicly available network administration tool plink to configure port-forwarding rules on compromised machines, enabling remote access via the remote desktop protocol (rdp). there is also ev

KeyboardInterrupt: ignored

In [ ]:
from IOC_EX.iocext import ioc_find
import json

if __name__ == '__main__':
    list_of_att_ioc = []
    with open('data/symantec.json') as json_file:
        data = json.load(json_file)
    for att in data:
        iocs = ioc_find(att['Explanation'])
        for key,value in iocs.items():
           if len(value) == 0 :
               del key
        list_of_att_ioc.append({
            'tite' : att['tite'],
            'Explanation' : att['Explanation'],
            'ioc' : iocs
        })

    json_object = json.dumps(list_of_att_ioc, indent=4,ensure_ascii=False)

    with open("/content/drive/MyDrive/symantec_ioc.json", "a") as outfile:
        outfile.write(json_object)






In [ ]:
import requests
import json
import random
import time
f = open('/content/drive/MyDrive/symantec.json', )

# returns JSON object as
# a dictionary
data = json.load(f)
lis = []
for att in data:
    body = {'data': att['Explanation']}
    try:
        r = requests.post('https://api.iocparser.com/raw', data=body,
                        proxies={'http': f'localhost:{random.randint(5002, 5016)}',
                                'https': f'localhost:{random.randint(5002, 5016)}'})
    except Exception :
        r = requests.post('https://api.iocparser.com/raw', data=body,
                        proxies={'http': f'localhost:{random.randint(5002, 5016)}',
                                'https': f'localhost:{random.randint(5002, 5016)}'})
    try:
        response = json.loads(r.text)
    except Exception:
        continue
    pp = {
        'tite':att['tite'],
        'Explanation' : att['Explanation'],
        'IOC' : response['data']
    }
    print(pp)
    lis.append(pp)
json_object = json.dumps(lis, indent=4,ensure_ascii=False)
with open("./data/symantec_iocparser.json", "a") as outfile:
    outfile.write(json_object)


In [ ]:
from IOC_EX.iocext import ioc_find
import json
import re


if __name__ == '__main__':
    list_of_att_ioc = []
    with open('/content/drive/MyDrive/symantec.json') as json_file:
        data = json.load(json_file)
    for att in data:
        cves = CVE = re.findall("cve-[0-9]{4}-[0-9]{4,6}",att['Explanation'])
        if len(cves) != 0:
            list_of_att_ioc.append({
                'tite' : att['tite'],
                'Explanation' : att['Explanation'],
                'ioc' : cves
            })

    json_object = json.dumps(list_of_att_ioc, indent=4,ensure_ascii=False)

    with open("/content/drive/MyDrive/symantec_cve.json", "a") as outfile:
        outfile.write(json_object)






In [ ]:
import concurrent.futures
import requests

def get_trending_tokens(api_key):
    url = 'https://min-api.cryptocompare.com/data/top/totalvolfull'
    params = {
        'limit': 1000,  # You can adjust the limit as per your requirement
        'tsym': 'USD',  # You can change the currency if needed
        'api_key': api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        tokens = data['Data']
        trending_tokens = [token['CoinInfo']['Name'] for token in tokens]
        return trending_tokens
    except requests.RequestException as e:
        print("Error fetching trending tokens:", e)
        return None

def main():
    # Replace 'YOUR_API_KEY' with your actual API key
    api_key = 'YOUR_API_KEY'

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(get_trending_tokens, api_key)
        trending_tokens = future.result()

    if trending_tokens:
        print("Trending Tokens:")
        for token in trending_tokens:
            print(token)

if __name__ == "__main__":
    main()

In [ ]:
import requests

# Replace 'YOUR_API_KEY' with your actual API key
api_key = 'import concurrent.futures
import requests

def get_trending_tokens(api_key):
    url = 'https://min-api.cryptocompare.com/data/top/totalvolfull'
    params = {
        'limit': 1000,  # You can adjust the limit as per your requirement
        'tsym': 'USD',  # You can change the currency if needed
        'api_key': api_key
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        tokens = data['Data']
        trending_tokens = [token['CoinInfo']['Name'] for token in tokens]
        return trending_tokens
    else:
        print("Error:", response.status_code)
        return None

def main():
    # Replace 'YOUR_API_KEY' with your actual API key
    api_key = 'YOUR_API_KEY'

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(get_trending_tokens, api_key)
        trending_tokens = future.result()

    if trending_tokens:
        print("Trending Tokens:")
        for token in trending_tokens:
            print(token)

if __name__ == "__main__":
    main()
'

# Example URLs
price_url = f'https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD&api_key={api_key}'
news_url = f'https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&api_key={api_key}'

# Example requests
price_response = requests.get(price_url)
news_response = requests.get(news_url)

print("Price URL:", price_url)
print("News URL:", news_url)

In [ ]:
import concurrent.futures
import requests

def get_trending_tokens(api_key):
    url = 'https://min-api.cryptocompare.com/data/top/totalvolfull'
    params = {
        'limit': 1000,  # You can adjust the limit as per your requirement
        'tsym': 'USD',  # You can change the currency if needed
        'api_key': api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        tokens = data['Data']
        trending_tokens = [token['CoinInfo']['Name'] for token in tokens]
        return trending_tokens
    except requests.RequestException as e:
        print("Error fetching trending tokens:", e)
        return None

def main():
    # Replace 'YOUR_API_KEY' with your actual API key
    api_key = 'a2c18ffbc00c5d578b712023df679c278bcc15aac63051d194062addc23968c7'

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(get_trending_tokens, api_key)
        trending_tokens = future.result()

    if trending_tokens:
        print("Trending Tokens:")
        for token in trending_tokens:
            print(token)

if __name__ == "__main__":
    main()

In [ ]:
import requests

# Replace 'YOUR_API_KEY' with your actual API key
api_key = 'a2c18ffbc00c5d578b712023df679c278bcc15aac63051d194062addc23968c7'

# Example URLs
price_url = f'https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD&api_key={api_key}'
news_url = f'https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&api_key={api_key}'

# Example requests
price_response = requests.get(price_url)
news_response = requests.get(news_url)

print("Price URL:", price_url)
print("News URL:", news_url)

Price URL: https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD&api_key=a2c18ffbc00c5d578b712023df679c278bcc15aac63051d194062addc23968c7
News URL: https://min-api.cryptocompare.com/data/v2/news/?categories=BTC&api_key=a2c18ffbc00c5d578b712023df679c278bcc15aac63051d194062addc23968c7
